In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Data+Scientist%22&l=USA&start={}"
max_results=3000
Linkdf=[]

for start in range(0, max_results, 10):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

2751

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

2751

# Looping through websites....

In [5]:
Title=[]
Descriptions=[]
Location=[]

for url in websites:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    locations=str(location_containers.text)
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)

# Extracting what we want from the Descriptions Column

In [6]:
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement'])

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]

In [7]:
Descriptions_df

,Title,Location,Qualifications,Skills,Responsibilities,Education,Requirement
0,Entry Level Data Scientist,United States,None Found,None Found,"\nYou will implement and validate predictive models as well as create and maintain statistical models with a focus on big data.\nYou will be exposed to and incorporate a variety of statistical and machine learning techniques such as logistic regression, experimental design, generalized linear models, mixed modeling, CHAID/decision trees, neural networks and ensemble models.\nYou’ll communicate with internal and external clients to understand business needs and provide analytical solutions.\nYou will use leading edge tools such as COGNOS, Watson Studio and Watson Machine Learning.\nYou’ll work in an Agile, collaborative environment, partnering with other scientists, engineers, and database administrators of all backgrounds and disciplines to bring analytical rigor and statistical methods to the challenges of predicting behaviors.",None Found,None Found
1,Entry Level - Associate Data Scientist,United States,None Found,None Found,"Ability to look at things differently, debug, troubleshoot, design and implement solutions to complex technical issues.\nStrong technical and analytical abilities, a knack for driving impact and growth, and experience with a programming/scripting in a language such as Java or Python.\nBasic understanding of statistical programming in a language such as R, Python, or SAS, SPSS, MATLAB.\nBasic understanding of Cloud (AWS, Azure, etc.)\nExcellent verbal and written communication skills.\nWork or internship experience using data science tools in a corporate environment.\nInterest in, understanding of, or experience with Design Thinking and Agile Development Methodologies\nWillingness to travel up to 100% of the time.",None Found,None Found
2,Jr. Data Scientist,"New York, NY 10036","Bachelor’s or Master’s degree in a quantitative field\nKnowledge of Python and/or R\nKnowledge of algorithms for data mining, machine learning, and/or natural language processing\nPossess some understanding of statistical procedures used in advanced analytics\nExperience processing large amounts of structured and unstructured data using Spark, Hive or other Big Data technologies\nExperience building scalable data models and performing complex relational database queries using SQL (Oracle, MySQL), etc.",None Found,"\nIdentify, ingest, and enrich a wide range of structured and unstructured data into datasets for analysis\nLearn to apply statistical and advanced techniques (e.g. segmentation, machine learning) to develop both prototypes and scalable (i.e. efficient, automated) data analyses\nConduct basic ad hoc analysis, building models, forecasting viewing behavior\nExtensive code development, debugging, optimizing, and productionizing\nSupport the development of machine learning pipeline architecture\nHelp build out frameworks for quickly rolling out new data analysis for standalone data-driven products and services to support our network of media brands.\nBe able to transform unstructured raw data in to formats suitable for modeling\nBe able to work in teams and collaborate with stakeholders to define requirements",None Found,None Found
3,Intern Conversion - Data Scientist,United States,None Found,None Found,"\nYou will implement and validate predictive models as well as create and maintain statistical models with a focus on big data.\nYou will be exposed to and incorporate a variety of statistical and machine learning techniques such as logistic regression, experimental design, generalized linear models, mixed modeling, CHAID/decision trees, neural networks and ensemble models.\nYou’ll communicate with internal and external clients to understand business needs and provide analytical solutions.\nYou will use leading edge tools such as COGNOS, Watson Studio and Watson Machine Learning.\nYou’ll work in an Agile, collaborative environment, partnering with other scientists, engineers, and dat

# Move to csv

In [8]:
Descriptions_df.to_csv('Descriptions_df_DataScientist.csv')